# 第一步 构造验证集

在机器学习模型（特别是深度学习模型）的训练过程中，模型是非常容易过拟合的。深度学习模型在不断的训练过程中训练误差会逐渐降低，但测试误差的走势则不一定。

在模型的训练过程中，模型只能利用训练数据来进行训练，模型并不能接触到测试集上的样本。因此模型如果将训练集学的过好，模型就会记住训练样本的细节，导致模型在测试集的泛化效果较差，这种现象称为过拟合（Overfitting）。与过拟合相对应的是欠拟合（Underfitting），即模型在训练集上的拟合效果较差。

验证集的划分有如下几种方式:

留出法（Hold-Out）
直接将训练集划分成两部分，新的训练集和验证集。这种划分方式的优点是最为直接简单；缺点是只得到了一份验证集，有可能导致模型在验证集上过拟合。留出法应用场景是数据量比较大的情况。

交叉验证法（Cross Validation，CV）
将训练集划分成K份，将其中的K-1份作为训练集，剩余的1份作为验证集，循环K训练。这种划分方式是所有的训练集都是验证集，最终模型验证精度是K份平均得到。这种方式的优点是验证集精度比较可靠，训练K次可以得到K个有多样性差异的模型；CV验证的缺点是需要训练K次，不适合数据量很大的情况。

自助采样法（BootStrap）
通过有放回的采样方式得到新的训练集和验证集，每次的训练集和验证集都是有区别的。这种划分方式一般适用于数据量较小的情况。



# 第二步 模型训练于验证
每轮进行训练和验证，并根据最优验证集精度保存模型。

In [3]:
import torch
import torch.nn as nn
from torch.utils.data.dataset import Dataset

# class SVHN_Model1(nn.Module):
#     def __init__(self):
#         super(SVHN_Model1, self).__init__()
                
#         model_conv = models.resnet18(pretrained=True)
#         model_conv.avgpool = nn.AdaptiveAvgPool2d(1)
#         model_conv = nn.Sequential(*list(model_conv.children())[:-1])
#         self.cnn = model_conv
        
#         self.fc1 = nn.Linear(512, 11)
#         self.fc2 = nn.Linear(512, 11)
#         self.fc3 = nn.Linear(512, 11)
#         self.fc4 = nn.Linear(512, 11)
#         self.fc5 = nn.Linear(512, 11)
    
#     def forward(self, img):        
#         feat = self.cnn(img)
#         # print(feat.shape)
#         feat = feat.view(feat.shape[0], -1)
#         c1 = self.fc1(feat)
#         c2 = self.fc2(feat)
#         c3 = self.fc3(feat)
#         c4 = self.fc4(feat)
#         c5 = self.fc5(feat)
#         return c1, c2, c3, c4, c5
    
train_loader = torch.utils.data.DataLoader(
#     train_dataset,
    batch_size=10, 
    shuffle=True, 
    num_workers=10, 
)
    
val_loader = torch.utils.data.DataLoader(
#     val_dataset,
    batch_size=10, 
    shuffle=False, 
    num_workers=10, 
)

model = SVHN_Model1()
criterion = nn.CrossEntropyLoss (size_average=False)
optimizer = torch.optim.Adam(model.parameters(), 0.001)
best_loss = 1000.0
for epoch in range(20):
    print('Epoch: ', epoch)

    train(train_loader, model, criterion, optimizer, epoch)
    val_loss = validate(val_loader, model, criterion)
    
    # 记录下验证集精度
    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), './model.pt')

TypeError: __init__() missing 1 required positional argument: 'dataset'

In [4]:
# 每一个epoch的训练代码
def train(train_loader, model, criterion, optimizer, epoch):
    # 切换模型为训练模式
    model.train()

    for i, (input, target) in enumerate(train_loader):
        c0, c1, c2, c3, c4, c5 = model(data[0])
        loss = criterion(c0, data[1][:, 0]) + \
                criterion(c1, data[1][:, 1]) + \
                criterion(c2, data[1][:, 2]) + \
                criterion(c3, data[1][:, 3]) + \
                criterion(c4, data[1][:, 4]) + \
                criterion(c5, data[1][:, 5])
        loss /= 6
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [5]:
# 每一个epoch的验证代码
def validate(val_loader, model, criterion):
    # 切换模型为预测模型
    model.eval()
    val_loss = []

    # 不记录模型梯度信息
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
            c0, c1, c2, c3, c4, c5 = model(data[0])
            loss = criterion(c0, data[1][:, 0]) + \
                    criterion(c1, data[1][:, 1]) + \
                    criterion(c2, data[1][:, 2]) + \
                    criterion(c3, data[1][:, 3]) + \
                    criterion(c4, data[1][:, 4]) + \
                    criterion(c5, data[1][:, 5])
            loss /= 6
            val_loss.append(loss.item())
    return np.mean(val_loss)

In [6]:
# 模型保存和加载
torch.save(model_object.state_dict(), 'model.pt')
model.load_state_dict(torch.load(' model.pt')) 

NameError: name 'model_object' is not defined

# 模型调参
1.采用学习率衰减的方式逐步调节学习率
2.采用warm up预热